In [1]:
from notebook_setup import setup_notebook
notebook_data = setup_notebook()

import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import fiona
from pathlib import Path
import sys
vergelijken_tool = sys.path.append(r"E:\personen\jacosta\Juan_test_20230317")
test_vergelijken = sys.path.append(r'E:\personen\jacosta\Juan_test_20230317\Test_vergelijkingsTool')
import importlib.resources as pkg_resources  # Load resource from package
import ipywidgets as widgets
from hhnk_threedi_tools.core.folders import Folders
from IPython.display import display
from ipywidgets import Button, HBox, VBox, Label
import ipywidgets as widgets
from Test_vergelijkingsTool import *
from Test_vergelijkingsTool import name_date
from Test_vergelijkingsTool import main
from Test_vergelijkingsTool import config
from datamodels import DAMO
from datamodels import Dataset
from datamodels import Threedimodel


In [ ]:
path  = name_date.path 
model_name, source_data, folder = name_date.name(path)

# polder polygon. It should be a geopackge file 
fn_DAMO_selection = name_date.damo_selection

#Base folder initial files. 
source_data_old = name_date.source_data_old

# Old DAMO (DCMB/FME export) and last version location . 
fn_damo_old = name_date.fn_damo_old
fn_damo_new = name_date.fn_damo_new
fn_damo_old_translation =  Path(os.path.join(source_data_old, 'damo_translation.json'))
fn_damo_new_translation = fn_damo_old_translation

#fn_damo_new_translation = Path(os.path.join(source_data_old, 'damo_translation.json'))

# Old HDB (DCMB/FME export)and last version location . 
fn_hdb_old = name_date.fn_hdb_old
fn_hdb_new = name_date.fn_hdb_new
fn_hdb_old_translation =Path(os.path.join(source_data_old, 'hdb_translation.json'))
fn_hdb_new_translation = fn_hdb_old_translation 

#3di model location file
fn_threedimodel = name_date.fn_threedimodel


json_file = os.path.join(source_data, 'vergelijkingsTool', 'json_files') 
fn_threedimodel_translation = Path(os.path.join(json_file, 'threedi_translation.json'))
fn_damo_attribute_comparison = Path(os.path.join(json_file, 'damo_attribute_comparison.json'))
fn_model_attribute_comparison = Path(os.path.join(json_file, 'model_attribute_comparison.json'))

# Define path where layer stylings can be found (for each layer it will search for <<LAYER_NAME>>.qml
styling_path = Path(os.path.join(source_data, 'styling'))

#output location. 
out_put_files = os.path.join(source_data, 'vergelijkingsTool', 'output') 
# Define outputs
fn_DAMO_comparison_export = Path(os.path.join(out_put_files, 'DAMO_comparison_Test_32.gpkg'))
fn_threedi_comparison_export = Path(os.path.join(out_put_files, 'Threedi_comparison_Test_25.gpkg'))

print(path)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
# Function to get the labels of selected ToggleButtons
def item_layout(width="95%", grid_area="", **kwargs):
    return widgets.Layout(
        width=width, grid_area=grid_area, **kwargs
    ) 
def get_selected_check_box(check_box):
    value = check_box.value
    return(value)

# Function to get the labels of selected ToggleButtons
def get_selected_toggle_buttons(toggle_buttons):
    items =  []
    for button in toggle_buttons:
        if button.value:
            items.append(button.description)
    return(items)

# Function to get the labels of selected ToggleButtons
def get_selection(toggle_buttons):
    for button in toggle_buttons:
        if button.value:
            return button.description
    return None

select_layer = widgets.Checkbox(
    value=False,
    description='Do you want to compare specific layer DAMO/DAMO',
    disabled=False,
    indent=False,
    layout = item_layout(width='50%')
)

select_layer_3di = widgets.Checkbox(
    value=False,
    description='Do you want to compare specific layer DAMO/3di',
    disabled=False,
    indent=False,
    layout = item_layout(width='50%')

)
# display(select_layer, select_layer_3di)

compare_title = Label(value = "Which database do you want to compare with?")
options = ['Compare with Damo', 'Compare with 3Di', 'Both']
compare_options =[widgets.ToggleButton(description = layer) for layer in options] 
compare_selection = VBox([compare_title, HBox([compare_options[0], compare_options[1], compare_options[2]])]) 
display(HBox([compare_selection]))


In [ ]:
select_compare = get_selection(compare_options)
if select_compare == 'Compare with Damo':
    display(select_layer)
elif select_compare == 'Compare with 3Di':
    display(select_layer_3di)
elif select_compare == 'Both':
    display(select_layer, select_layer_3di)
else:
    print('You must select only one layer')



# compare_threedi = get_selected_compare_wtih(compare_options)
# compare_damo = get_selected_toggle_buttons(compare_with_damo)



In [ ]:
layer_selection = get_selected_check_box(select_layer)
threedi_layer_selector = get_selected_check_box(select_layer_3di)
print(f'DAMO layer_selection is set as {layer_selection}, 3di layer selection is set as {threedi_layer_selector}.\
 We are compering with {select_compare}')

layers_damo_new = fiona.listlayers(fn_damo_new)
layer_hdb_new = fiona.listlayers(fn_hdb_old)

if layer_selection == True:
    
    threedi_structure_selection = False
    damo_new_title = Label(value = "Select Layer that you want to compare with DAMO")
    HDB_new_title = Label(value = "Select Layer that you want to compare with HDB")

    damo_new_items = [widgets.ToggleButton(description = layer, layout = item_layout(width='50%')) for layer in layers_damo_new] 
    hdb_new_items = [widgets.ToggleButton(description = layer, layout = item_layout(width='50%')) for layer in layer_hdb_new] 

    left_box = VBox([damo_new_items[0], damo_new_items[1], damo_new_items[2], damo_new_items[3], damo_new_items[4], damo_new_items[5], damo_new_items[6], damo_new_items[7], damo_new_items[8]])
    right_box = VBox([damo_new_items[9], damo_new_items[10], damo_new_items[11], damo_new_items[12], damo_new_items[13], damo_new_items[14], damo_new_items[15], damo_new_items[16], damo_new_items[17]])

    hdb_left_box_new = VBox([hdb_new_items[0], hdb_new_items[1], hdb_new_items[2], hdb_new_items[3]])
    hdb_right_box_new = VBox([hdb_new_items[4], hdb_new_items[5], hdb_new_items[6], hdb_new_items[7]])

    damo_new = VBox([damo_new_title, HBox([left_box, right_box])])
    hdb_new = VBox([HDB_new_title, HBox([hdb_left_box_new, hdb_right_box_new])])
    # damo_old = VBox([damo_old_title, HBox([left_box_old, right_box_old])])
    # hdb_old = VBox([HDB_old_title, HBox([hdb_left_box_old, hdb_right_box_old])])

    display(HBox([damo_new, hdb_new]))
# HBox([left_box, right_box])

    # layers_input_hdb_selection = get_selected_toggle_buttons(hdb_new_items)
    # layers_input_damo_selection = get_selected_toggle_buttons(damo_new_items)
else:
    layers_damo_new = layers_damo_new
    layer_hdb_new  = layer_hdb_new


if threedi_layer_selector == True:
    
    layer_selection = False
    structure_new_title = Label(value = "Select The Structure to compare with")
    HDB_new_title = Label(value = "Select Layer that you want to compare with 3di")
    thredi_new_title = Label(value = "Select Layer that you want to compare with DAMO/HDB")


    threedi_structure_selection = [widgets.ToggleButton(description = layer, layout = item_layout(width='95%')) for layer in config.THREEDI_STRUCTURE_LAYERS] 
    damo_structure_selection = [widgets.ToggleButton(description = layer, layout = item_layout(width='95%')) for layer in config.DAMO_HDB_STRUCTURE_LAYERS] 
    structure_codes = [widgets.ToggleButton(description = layer, layout = item_layout(width='95%')) for layer in config.STRUCTURE_CODES] 

    left_box = VBox([threedi_structure_selection[0], threedi_structure_selection[1], threedi_structure_selection[2]])
    right_box = VBox([threedi_structure_selection[3], threedi_structure_selection[4]])

    hdb_left_box_new = VBox([damo_structure_selection[0], damo_structure_selection[1], damo_structure_selection[2], damo_structure_selection[3]])
    hdb_right_box_new = VBox([damo_structure_selection[4], damo_structure_selection[5], damo_structure_selection[6], damo_structure_selection[7]])

    structure_lef_box  = VBox([structure_codes[0], structure_codes[1], structure_codes[2], structure_codes[3]])
    structure_right_box  = VBox([structure_codes[4], structure_codes[5], structure_codes[6]])

    damo_new = VBox([thredi_new_title, HBox([left_box, right_box])])
    hdb_new = VBox([HDB_new_title, HBox([hdb_left_box_new, hdb_right_box_new])])
    structure_new = VBox([structure_new_title, HBox([structure_lef_box, structure_right_box])])
    # damo_old = VBox([damo_old_title, HBox([left_box_old, right_box_old])])
    # hdb_old = VBox([HDB_old_title, HBox([hdb_left_box_old, hdb_right_box_old])])

    display(VBox([damo_new, hdb_new, structure_new]),)
# HBox([left_box, right_box])

    # layers_input_hdb_selection = get_selected_toggle_buttons(hdb_new_items)
    # layers_input_damo_selection = get_selected_toggle_buttons(damo_new_items)
else:
    layers_damo_new = layers_damo_new
    layer_hdb_new  = layer_hdb_new

In [ ]:
if layer_selection == True:
    layers_input_hdb_selection = get_selected_toggle_buttons(hdb_new_items)
    layers_input_damo_selection = get_selected_toggle_buttons(damo_new_items)
    print(f'The layers selected from DAMO are {layers_input_damo_selection} and from HDB are {layers_input_hdb_selection}')
else:
    layers_input_hdb_selection = None
    layers_input_damo_selection = None
    
if threedi_layer_selector == True:
    threedi_structure_selection = get_selected_toggle_buttons(threedi_structure_selection)
    damo_structure_selection = get_selected_toggle_buttons(damo_structure_selection)
    structure_codes = get_selected_toggle_buttons(structure_codes)

    # print(f'Damo layer selector is {layer_selection}, 3di layer selector is {threedi_layer_selector}')
    print(f'The 3di layers selected from are {threedi_structure_selection}, from DAMOA and HDB the layers\
            are {damo_structure_selection}, the structure code to be used are {structure_codes}')
    # print(f'The layers selected from DAMO are {layers_input_damo_selection} and from HDB are {layers_input_hdb_selection}')
else:
    damo_structure_selection = []
    structure_codes = []
    print(f'Damo layer selector is {layer_selection}, 3di layer selector is {threedi_layer_selector}')
    

In [ ]:
main.main(fn_DAMO_selection,fn_damo_old_translation, fn_damo_new, fn_hdb_new, fn_damo_old, fn_hdb_old, fn_threedimodel,
        fn_threedimodel_translation, fn_damo_attribute_comparison, fn_model_attribute_comparison, styling_path, 
        fn_damo_new_translation, fn_DAMO_comparison_export, fn_threedi_comparison_export, select_compare, layer_selection, 
        layers_input_hdb_selection= layers_input_hdb_selection, layers_input_damo_selection= layers_input_damo_selection,
        threedi_layer_selector = threedi_layer_selector, threedi_structure_selection= threedi_structure_selection, 
        damo_structure_selection=damo_structure_selection,structure_codes = structure_codes
        )